## Lab. 12

### Solve the following problem using Genetic Algorithms:


Problem: Weighted N-Queen Problem


You are given an N×N chessboard, and each cell of the board has an associated weight. Your task is to find a valid placement of N queens such that the total weight of the queens is maximized, and no two queens threaten each other.





In the traditional N-Queen Problem, the goal is to place N queens on an N×N chessboard in such a way that no two queens threaten each other. In this variation, we introduce weights to the queens and aim to find a placement that maximizes the total weight of the queens while satisfying the constraint of non-threatening positions.


Constraints:

1. There should be exactly one queen in each row and each column.
2. No two queens should be placed in the same diagonal, i.e., they should not threaten each other.
3. The placement should maximize the total weight of the queens.


Representation:

Use a permutation-based representation. Each permutation represents the column position of the queen for each row. 

For example, if N=4, a valid permutation [2, 4, 1, 3] indicates that the queen in the first row is placed in column 2, the queen in the second row is placed in column 4, and so on.


Genetic Algorithm Steps:

1. *Initialization*: Generate an initial population of permutations randomly.

2. *Fitness Evaluation*: Evaluate the fitness of each permutation by calculating the total weight of the queens while considering the non-threatening positions.

3. *Selection*: Select a subset of permutations from the population based on their fitness, using selection techniques like tournament selection or roulette wheel selection.

4. *Crossover*: Perform crossover (recombination) on the selected permutations to create new offspring permutations.

5. *Mutation*: Introduce random changes (mutations) in the offspring permutations to maintain diversity in the population.

6. *Fitness Evaluation for the new individuals*: Evaluate the fitness of the new population.

7. *Form the new population*: Select the surviving individuals based on scores, with chances direct proportional with their performance.

8. Repeat steps 3-7 for a certain number of generations or until a termination condition is met (e.g., a maximum number of iterations or a satisfactory solution is found).


9. *Termination*: Return the best-performing individual (permutation) found as the solution to the problem.

Note: The fitness function used in this problem should calculate the total weight of the queens based on the positions specified by the permutation. Additionally, the fitness function should penalize solutions that violate the non-threatening constraint by assigning a lower fitness score to such permutations.

In [5]:
import random

def initialize_permutation(n):
    return random.sample(range(1, n+1), n)

def initialize_population(population_size, n):
    return [initialize_permutation(n) for _ in range(population_size)]

In [7]:
population_size = 10
n = 5
population = initialize_population(population_size, n)
print(population)

[[4, 1, 5, 2, 3], [4, 5, 3, 1, 2], [2, 1, 4, 3, 5], [4, 2, 3, 1, 5], [4, 2, 5, 3, 1], [2, 5, 3, 4, 1], [5, 4, 2, 1, 3], [5, 2, 1, 4, 3], [5, 2, 4, 3, 1], [5, 4, 1, 3, 2]]


In [9]:
def fitness(individual, weights):
    n = len(individual)
    total_weight = 0
    penalty = 0

    for i in range(n):
        total_weight += weights[i][individual[i]-1]

    for i in range(n):
        for j in range(i+1, n):
            # Check if queens are in the same diagonal
            if abs(individual[i] - individual[j]) == abs(i - j):
                penalty += 1

    for i in range(n):
        for j in range(i+1, n):
            if individual[i] == individual[j]:
                penalty += 1

    # Subtract penalty from total weight
    fitness_score = total_weight - penalty

    return fitness_score

In [12]:
weights = [[1, 2, 3, 4, 5], [5, 4, 3, 2, 1], [1, 3, 5, 2, 4], [4, 2, 5, 3, 1], [3, 5, 2, 4, 1]]
individual = [2, 4, 1, 3, 5]
fitness_score = fitness(individual, weights)
print(fitness_score)
individual = [1, 2, 3, 4, 5]
fitness_score = fitness(individual, weights)
print(fitness_score)

11
4


In [13]:
def evaluate_fitness(population, weights):
    return [fitness(individual, weights) for individual in population]

In [15]:
fitness_scores = evaluate_fitness(population, weights)
print(fitness_scores)

[15, 15, 11, 14, 20, 12, 14, 11, 17, 16]


In [16]:
def select_parents(population, fitness_scores, num_parents, k=2):
    parents = []
    for _ in range(num_parents):
        # Select k individuals for the tournament
        tournament_individuals = random.choices(population, k=k)
        # Get their fitness scores
        tournament_scores = [fitness_scores[population.index(individual)] for individual in tournament_individuals]
        # Select the individual with the highest fitness
        winner = tournament_individuals[tournament_scores.index(max(tournament_scores))]
        # Add the winner to the parents list
        parents.append(winner)
    return parents

In [18]:
num_parents = 2
parents = select_parents(population, fitness_scores, num_parents)
print(parents)

[[5, 4, 2, 1, 3], [4, 1, 5, 2, 3]]


In [19]:
def crossover(parents, num_offspring):
    offspring = []
    for _ in range(num_offspring):
        # Select two parents randomly
        parent1, parent2 = random.sample(parents, 2)
        # Select a random crossover point
        crossover_point = random.randint(1, len(parent1)-1)
        # Perform crossover
        child = parent1[:crossover_point] + parent2[crossover_point:]
        offspring.append(child)
    return offspring

In [20]:
num_offspring = 2
offspring = crossover(parents, num_offspring)
print(offspring)

[[5, 1, 5, 2, 3], [5, 4, 2, 2, 3]]


In [21]:
def mutation(individual, mutation_rate):
    n = len(individual)
    mutated_individual = individual.copy()
    for i in range(n):
        if random.random() < mutation_rate:
            # Generate a random new position for the queen
            new_position = random.randint(1, n)
            mutated_individual[i] = new_position
    return mutated_individual

In [27]:
mutation_rate = 0.1
individual = [2, 4, 1, 3, 5]
mutated_individual = mutation(individual, mutation_rate)
print(mutated_individual)

[2, 2, 1, 2, 5]


In [29]:
def select_new_population(population, fitness_scores, num_survivors):
    # Sort the population based on fitness scores
    sorted_population = [x for _, x in sorted(zip(fitness_scores, population), key=lambda pair: pair[0], reverse=True)]
    # Select the top individuals as survivors
    new_population = sorted_population[:num_survivors]
    return new_population

In [32]:
def genetic_algorithm(population_size, n, weights, num_generations, num_parents=2, num_offspring=2, mutation_rate=0.1):
    # Initialize the population
    population = initialize_population(population_size, n)
    for _ in range(num_generations):
        # Evaluate fitness of the population
        fitness_scores = evaluate_fitness(population, weights)
        # Select parents
        parents = select_parents(population, fitness_scores, num_parents)
        # Perform crossover
        offspring = crossover(parents, num_offspring)
        # Perform mutation
        mutated_offspring = [mutation(child, mutation_rate) for child in offspring]
        # Evaluate fitness of the new population
        offspring_fitness = evaluate_fitness(mutated_offspring, weights)
        # Combine the old population and the new population
        combined_population = population + mutated_offspring
        combined_fitness = fitness_scores + offspring_fitness
        # Select the new population
        population = select_new_population(combined_population, combined_fitness, population_size)
    return population

In [35]:
def select_best_individual(population, weights):
    fitness_scores = evaluate_fitness(population, weights)
    best_individual = population[fitness_scores.index(max(fitness_scores))]
    return best_individual, max(fitness_scores)

In [36]:
population_size = 10
n = 5
weights = [[1, 2, 3, 4, 5], [5, 4, 3, 2, 1], [1, 3, 5, 2, 4], [4, 2, 5, 3, 1], [3, 5, 2, 4, 1]]
num_generations = 100
population = genetic_algorithm(population_size, n, weights, num_generations)

In [37]:
best_individual, best_fitness = select_best_individual(population, weights)
print(best_individual, best_fitness)


[5, 1, 3, 4, 2] 21
